# MBTI별 포스팅 모으기 

In [4]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [5]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# tokenize and stemming

In [6]:
X_data = mbti['posts'].ravel()
y_data = mbti['type']


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [7]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ["http*"]
# print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
print(stemData[1])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['i', 'do', 'stuff', 'like', 'disthey', 'no', 'longer', 'becom', 'outlandish', 'onc', 'they', 'start', 'becom', 'realiti', 'i', 'think', 'you', 'also', 'have', 'to', 'alway', 'rememb', 'whi', 'you', 'start', 'to', 'make', 'music', 'in', 'the', 'first', 'place', 'the', 'way', 'it', 'feel', 'or', 'how', 'you', 'make', 'othersim', 'a', 'musician', 'myself', 'but', 'i', 'start', 'record', 'when', 'i', 'wa', 'a', 'junior', 'in', 'high', 'school', 'i', 'start', 'write', 'music', 'that', 'would', 'person', 'appeal', 'to', 'me', 'and', 'honestli', 'it', 'actual', 'work', 'out', 'pretti', 'well', 'sothen', 'again', 'i', 'hate', 'to', 'gener', 'but', 'that', 'wa', 'my', 'experienceboth', 'loldat', 'an', 'esfj', 'wa', 'my', 'last', 'girl', 'but', 'she', 'wa', 'crazi', 'bruh', 'veri', 'on', 'and', 'off', 'i', 'call', 'her', 'out', 'on', 'it', 'becaus', 'it', 'seem', 'like', 'she', 'wa', 'never', 'consist', 'on', 'how', 'she', 'actual', 'felt', 'but', 'it', 'cool', 'doe', 'n

143

In [9]:
#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
y = np.array(y_data)




[[0.15769379 0.         0.         ... 0.         0.         0.        ]
 [0.22235513 0.0403856  0.         ... 0.         0.         0.        ]
 [0.14209875 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.18852251 0.         0.         ... 0.         0.         0.        ]
 [0.19113407 0.         0.         ... 0.         0.         0.        ]
 [0.13241067 0.         0.         ... 0.         0.         0.        ]]


In [10]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
'''
X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)'''

Counter(y_data)
X_resampled, y_resampled = SMOTE(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)

Counter({'ENFJ': 1832,
         'ENFP': 1832,
         'ENTJ': 1832,
         'ENTP': 1832,
         'ESFJ': 1832,
         'ESFP': 1832,
         'ESTJ': 1832,
         'ESTP': 1832,
         'INFJ': 1832,
         'INFP': 1832,
         'INTJ': 1832,
         'INTP': 1832,
         'ISFJ': 1832,
         'ISFP': 1832,
         'ISTJ': 1832,
         'ISTP': 1832})

# XGBoost_Oversampling

In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [12]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
preds = model.predict(X_test)
print("Model building End")

Model building End


In [14]:
import joblib

joblib.dump(model, 'XGBoost_SMOTEoversample.pkl')

['XGBoost_oversample.pkl']

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.9388219240391176
[0.96068376 0.93895871 0.9751773  0.90893471 0.99613153 0.99626168
 0.99480069 0.98345588 0.8313253  0.81782178 0.89421158 0.85480944
 0.97913043 0.93526405 0.9754717  0.9556314 ]
[0.99118166 0.93392857 1.         0.93628319 1.         1.
 1.         1.         0.76243094 0.72202797 0.85823755 0.8365897
 1.         0.99637024 1.         0.9929078 ]
[0.97569444 0.93643688 0.98743268 0.92240628 0.99806202 0.99812734
 0.99739357 0.99165894 0.79538905 0.76694522 0.87585533 0.84560144
 0.98945518 0.96485062 0.98758357 0.97391304]
Evaluation END
